In [0]:
import pandas as pd
import numpy as np
from scipy.stats import norm

### bond pricing
bonds = pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/bonds.csv")

# drop rows with remaining coupon = 0 to avoid devided by 0 issues
bonds = bonds[bonds.CouponsRemaining != 0]

bonds.YearlyRate = bonds.YearlyRate / 100
# principal is in cents
bonds.Principal = bonds.Principal/100

# N is the total number of coupon payment
bonds = bonds.assign(N = bonds.CouponsRemaining+bonds.CurrentCoupon)
# delta_t is the length of coupon period in days
bonds = bonds.assign(delta_t = 360 * bonds.Time2Maturity / bonds.CouponsRemaining)
# M is the number of coupon payments per year
bonds = bonds.assign(M = 360 / bonds.delta_t)
# C is coupon amount
bonds = bonds.assign(C = bonds.Principal * bonds.YearlyRate / bonds.M)
# convert PD from string to float and apply a floor of 0.03%
bonds.PD = bonds.PD.apply(lambda x: x.strip('%'))
bonds.PD = bonds.PD.astype(float)/100
bonds.loc[bonds.PD < 0.0003, "PD"] = 0.0003
# t is total maturity year
bonds = bonds.assign(t = bonds.N * bonds.delta_t / 360)
# y is the market yield quoted at Oct 2 with term structure
bonds = bonds.assign(y = 0.0147)
bonds.loc[bonds.t < 10,"y"] = 0.0132
bonds.loc[bonds.t < 5,"y"] = 0.0137
bonds.loc[bonds.t < 3,"y"] = 0.015

# today_price 
def calculate_price(y,C,n,F):
  payment_series = np.repeat(C,n+1)
  payment_series[0] -= C
  payment_series[-1] += F
  price = np.npv(y,payment_series)
  return price

bonds = bonds.assign(today_price = bonds.apply(lambda x:calculate_price(x.y,x.C,x.CouponsRemaining,x.Principal),axis=1)) 

### calculate capital requirement
#calculate correlation R
bonds = bonds.assign(R = 0.12*(1-np.exp(-50*bonds.PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*bonds.PD))/(1-np.exp(-50))))

#assume all bonds are subordinated,lgd = 0.45

#b is a parameter in calculating capital requirement
bonds = bonds.assign(b = (0.11852 - 0.05478 * np.log(bonds.PD))**2)

# get effective maturity of a bond using macaulay duration assuming interest rate = 0
def get_effective_duration(F,price,C,n):
  return (sum([ i*C for i in range(int(n))]) + n*F )/price
bonds = bonds.assign(effective_maturity = bonds.apply(lambda x:get_effective_duration(x.Principal,x.today_price,x.C,x.CouponsRemaining),axis=1))

# capital requirement
bonds = bonds.assign(capital_requirement = norm.cdf(np.sqrt((1-bonds.R) ** (-1))*norm.ppf(bonds.PD) + np.sqrt(bonds.R/(1-bonds.R)) * norm.ppf(0.999) )- bonds.PD )
bonds.capital_requirement = bonds.capital_requirement*(1+(bonds.effective_maturity - 2.5) * bonds.b)/(1 - 1.5 * bonds.b)
bonds.capital_requirement = bonds.capital_requirement * 0.45
# deal with na
bonds = bonds.fillna(0) 
# EAD
bonds = bonds.assign(EAD = bonds.BondsHeld * bonds.today_price)
# provision = PD * LGD * EAD
bonds = bonds.assign(provision = bonds.PD * bonds.EAD * 0.45)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
sum(bonds.capital_requirement)

212015.9059742439

In [0]:
sum(bonds.provision)

3183969185.577614

In [0]:
### calculate for mortgage
mortgage=pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/mortgage.csv")

# convert pd,lgd and interest rate from str to numeric value
mortgage.PD = mortgage.PD.apply(lambda x: x.strip('%'))
mortgage.PD = mortgage.PD.astype(float)/100
mortgage.LGD = mortgage.LGD.apply(lambda x: x.strip('%'))
mortgage.LGD = mortgage.LGD.astype(float)/100
mortgage.Rate = mortgage.Rate.apply(lambda x: x.strip('%'))
mortgage.Rate = mortgage.Rate.astype(float)/100

# apply pd floor 0.03%
mortgage.loc[mortgage.PD < 0.0003, "PD"] = 0.0003

# calculate the monthly payment
mortgage = mortgage.assign(monthly_pay= -1*np.pmt(mortgage.Rate/12,12*mortgage.TermYears, mortgage.OriginalAmount))
# N is the number of total installment
#mortgage = mortgage.assign(N = mortgage.TermYears * 12)

Formula to calculate monthly payment is:
$\sum_i^N\frac{C}{(1+\frac{r}{12})^i} = X$

Formula to calculate unpaid principal is:

$Unpaid Principal = X(1+\frac{r}{12})^N - C(\frac{(1+\frac{r}{12})^N - 1}{\frac{r}{12}})$


where r is yearly rate, N is the number of total installments,
C is monthly payment,
X is original amount


$new LGD = \frac{Unpaid Principal - collateral}{Unpaid Principal}$


In [0]:
# calculate unpaid principal
def get_unpaid_principal(C,N,r,OriginalAmount):
  # N is number of installments paid
  outstanding = OriginalAmount
  for i in range(int(N)):
    outstanding -= (C-outstanding*r/12)
  return outstanding
mortgage = mortgage.assign(unpaid_principal=mortgage.apply(lambda x:get_unpaid_principal(x.monthly_pay,x.CurrentInstallment,x.Rate,x.OriginalAmount),axis=1))
# collateral value should be discounted when calculating new LGD.
mortgage = mortgage.assign(newLGD = (mortgage.unpaid_principal - mortgage.Collateral_valuation) / mortgage.unpaid_principal )
# apply LGD 10% floor
mortgage.loc[mortgage.newLGD < 0.1,"newLGD"] = 0.1
# EAD = min(unpaid principal - collateral,0)
mortgage = mortgage.assign(EAD = mortgage.unpaid_principal - mortgage.Collateral_valuation)
mortgage.loc[mortgage.EAD < 0, "EAD"] = 0
### capital requirement. use R=0.15 for mortgage
mortgage = mortgage.assign(capital_requirement = norm.cdf(np.sqrt((1-.15)** (-1))*norm.ppf(mortgage.PD) + np.sqrt(.15/(1-.15)) * norm.ppf(0.999))-mortgage.PD )
mortgage.capital_requirement *= mortgage.newLGD
# deal with na
mortgage = mortgage.fillna(0)
# provision
mortgage = mortgage.assign(provision = mortgage.PD * mortgage.newLGD* mortgage.EAD) 

In [0]:
sum(mortgage.capital_requirement)

1566.2032272403326

In [0]:
sum(mortgage.provision)

536487966.90113604

In [0]:
### calculate for retail loans
retail = pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/retailloan.csv")
# convert PD from string to float and apply a floor of 0.03%
retail.PD = retail.PD.apply(lambda x: x.strip('%'))
retail.PD = retail.PD.astype(float)/100
retail.loc[retail.PD < 0.0003, "PD"] = 0.0003
#convert interest rate from str to float
retail.YearlyInterestRate = retail.YearlyInterestRate.apply(lambda x: x.strip('%'))
retail.YearlyInterestRate = retail.YearlyInterestRate.astype(float)/100
### capital requirement
# coefficient R
retail = retail.assign(R = 0.03*(1-np.exp(-35*retail.PD))/(1-np.exp(-35))+0.16*(1-(1-np.exp(-35*retail.PD))/(1-np.exp(-35))))
# monthly payment
retail = retail.assign(monthly_pay= -1*np.pmt(retail.YearlyInterestRate/12,retail.Term, retail.OriginalAmount)) 
# get outstanding payment
retail = retail.assign(unpaid_principal=retail.apply(lambda x:get_unpaid_principal(x.monthly_pay,x.CurrentInstallment,x.YearlyInterestRate,x.OriginalAmount),axis=1))
### capital requirement
# for retail loans, lgd = 100%
retail = retail.assign(capital_requirement = norm.cdf(np.sqrt((1-retail.R)** (-1))*norm.ppf(retail.PD) + np.sqrt(retail.R/(1-retail.R)) * norm.ppf(0.999) )- retail.PD )
# fill na with 0 if any
retail = retail.fillna(0)
# fill na with 0 if any
retail = retail.fillna(0)
# provision
retail = retail.assign(provision = retail.PD * retail.unpaid_principal)

In [0]:
sum(retail.capital_requirement)

2677.417925791526

In [0]:
sum(retail.provision)

25753734.835255027

In [0]:
### RWA
bonds = bonds.assign(rwa = 12.5*bonds.capital_requirement*bonds.EAD)
mortgage = mortgage.assign(rwa=12.5*mortgage.capital_requirement*mortgage.EAD)
retail= retail.assign(rwa=12.5*retail.capital_requirement*retail.unpaid_principal)
totalrwa = sum(bonds.rwa)+sum(mortgage.rwa)+sum(retail.rwa)
regcapital = 0.115 * totalrwa

In [0]:
regcapital

37380441880.20227